In [1]:
import pyphoplacecellanalysis
import pyqtgraph as pg
from pyqtgraph.Qt import QtCore, QtGui, QtWidgets
import pyqtgraph.opengl as gl # for 3D raster plot

import numpy as np

from pyphocorehelpers.general_helpers import OrderedMeta
from pyphocorehelpers.print_helpers import SimplePrintable, PrettyPrintable
from pyphocorehelpers.geometry_helpers import find_ranges_in_window

from pyphoplacecellanalysis.General.DataSeriesToSpatial import DataSeriesToSpatial

""" This notebook was to test the ability to generate a single-mesh representation of disconnected rectangular boxes, as would be done if trying to render the epochs along a timeline in 3D. 

Ultimately did not get this working, but switched to Vedo and did have success getting the epochs plotted.

"""

def _build_cube_mesh_data():
    vertexes = np.array([[1, 0, 0], #0
                        [0, 0, 0], #1
                        [0, 1, 0], #2
                        [0, 0, 1], #3
                        [1, 1, 0], #4
                        [1, 1, 1], #5
                        [0, 1, 1], #6
                        [1, 0, 1]])#7
    faces = np.array([[1,0,7], [1,3,7],
                    [1,2,4], [1,0,4],
                    [1,2,6], [1,3,6],
                    [0,4,5], [0,7,5],
                    [2,4,5], [2,6,5],
                    [3,6,5], [3,7,5]])
    colors = np.array([[1,0,0,1] for i in range(12)])
    md = gl.MeshData(vertexes=vertexes, faces=faces, edges=None, vertexColors=None, faceColors=colors)
    return md


def _build_separated_test_rects_mesh_data():
    vertexes = np.array([[1, 0, 0], #0
                        [0, 0, 0], #1
                        [0, 1, 0], #2
                        [0, 0, 1], #3
                        [1, 1, 0], #4
                        [1, 1, 1], #5
                        [0, 1, 1], #6
                        [1, 0, 1]])#7

    faces = np.array([[1,0,7], [1,3,7],
                    [1,2,4], [1,0,4],
                    [1,2,6], [1,3,6],
                    [0,4,5], [0,7,5],
                    [2,4,5], [2,6,5],
                    [3,6,5], [3,7,5]])

        
    
    shifted_verticies = vertexes.copy() + [3, 0, 0] # shift by x val of 3
    shifted_faces = np.shape(vertexes)[0] + faces.copy() # advance the indicies by the number of verticies so they make sense when the lists are appended.

    colors = np.array([[1,0,0,1] for i in range(12)])
    md = gl.MeshData(vertexes=vertexes, faces=faces, edges=None, vertexColors=None, faceColors=colors)
    return md

In [2]:
vertexes = np.array([[1, 0, 0], #0
                    [0, 0, 0], #1
                    [0, 1, 0], #2
                    [0, 0, 1], #3
                    [1, 1, 0], #4
                    [1, 1, 1], #5
                    [0, 1, 1], #6
                    [1, 0, 1]])#7

faces = np.array([[1,0,7], [1,3,7],
                [1,2,4], [1,0,4],
                [1,2,6], [1,3,6],
                [0,4,5], [0,7,5],
                [2,4,5], [2,6,5],
                [3,6,5], [3,7,5]])

num_original_verticies = np.shape(vertexes)[0]
num_original_faces = np.shape(faces)[0]

print(f'num_original_verticies: {num_original_verticies}, num_original_faces: {num_original_faces}')
shifted_verticies = vertexes.copy() + [3, 0, 0] # shift by x val of 3
shifted_faces = (num_original_verticies-1) + faces.copy() # advance the indicies by the number of verticies so they make sense when the lists are appended.

merged_verticies = np.vstack((vertexes, shifted_verticies))
merged_faces = np.vstack((faces, shifted_faces))
print(f'np.shape(merged_verticies): {np.shape(merged_verticies)}, np.shape(merged_faces): {np.shape(merged_faces)}')

num_merged_verticies = np.shape(merged_verticies)[0]
num_merged_faces = np.shape(merged_faces)[0]

merged_colors = np.array([[1,0,0,0.5] for i in range(num_merged_faces)])
merged_vertex_colors = np.vstack((np.array([[0,1,0,0.5] for i in range(num_original_faces)]), np.array([[0,0,1,0.5] for i in range(num_original_faces)])))
# colors = np.array([[1,0,0,1] for i in range(12)])
# md = gl.MeshData(vertexes=vertexes, faces=faces, edges=None, vertexColors=None, faceColors=colors)

md = gl.MeshData(vertexes=merged_verticies, faces=merged_faces, edges=None, vertexColors=merged_vertex_colors, faceColors=merged_colors)

num_original_verticies: 8, num_original_faces: 12
np.shape(merged_verticies): (16, 3), np.shape(merged_faces): (24, 3)


In [3]:
merged_colors

array([[1. , 0. , 0. , 0.5],
       [1. , 0. , 0. , 0.5],
       [1. , 0. , 0. , 0.5],
       [1. , 0. , 0. , 0.5],
       [1. , 0. , 0. , 0.5],
       [1. , 0. , 0. , 0.5],
       [1. , 0. , 0. , 0.5],
       [1. , 0. , 0. , 0.5],
       [1. , 0. , 0. , 0.5],
       [1. , 0. , 0. , 0.5],
       [1. , 0. , 0. , 0.5],
       [1. , 0. , 0. , 0.5],
       [1. , 0. , 0. , 0.5],
       [1. , 0. , 0. , 0.5],
       [1. , 0. , 0. , 0.5],
       [1. , 0. , 0. , 0.5],
       [1. , 0. , 0. , 0.5],
       [1. , 0. , 0. , 0.5],
       [1. , 0. , 0. , 0.5],
       [1. , 0. , 0. , 0.5],
       [1. , 0. , 0. , 0.5],
       [1. , 0. , 0. , 0.5],
       [1. , 0. , 0. , 0.5],
       [1. , 0. , 0. , 0.5]])

In [7]:
# app = QtGui.QApplication.instance()
# if app is None:
#     app = QtGui.QApplication([])

app = pg.mkQApp()

w = gl.GLViewWidget()
w.opts['distance'] = 20
w.show()
w.setWindowTitle('pyqtgraph example: GLScatterPlotItem')

g = gl.GLGridItem()
w.addItem(g)

# test_mesh_data = _build_separated_test_rects_mesh_data()
test_mesh_data = md
# test_mesh_data = _build_cube_mesh_data()
# test_mesh = gl.GLMeshItem(vertexes=vertexes, faces=faces, faceColors=colors, drawEdges=True, edgeColor=(0, 0, 0, 1))
test_mesh = gl.GLMeshItem(meshdata=test_mesh_data, smooth=True, drawEdges=True, edgeColor=(0, 0, 0, 1), draw=True)



w.addItem(test_mesh)

## Start Qt event loop unless running in interactive mode.
if __name__ == '__main__':
    import sys
    if (sys.flags.interactive != 1) or not hasattr(QtCore, 'PYQT_VERSION'):
        app.exec_()
        # QtGui.QApplication.instance().exec_()
        
        

C:\Users\pho\AppData\Local\Temp\ipykernel_2368\3208951805.py:29: DeprecationWarning: 'exec_' will be removed in the future. Use 'exec' instead.
  app.exec_()
c:\Users\pho\repos\Spike3DWorkEnv\Spike3D\.venv\lib\site-packages\pyqtgraph\opengl\MeshData.py:218: RuntimeWarning: invalid value encountered in divide
  norm /= (norm**2).sum()**0.5  ## and re-normalize
